<a href="https://colab.research.google.com/github/XxNessuxX/Shape-E-img23D-colab-in-spanish.-OpenAI/blob/main/Shap_E_img23D_en_espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="#97C145">**Shape-E img23D en español. (OpenAI)**


---


#### *Imagen a 3D - Muestra*


---


##### Cuaderno creado por: [Xx_Nessu_xX](https://fakeyou.com/)
##### Repositorio original de [Shape-E](https://github.com/openai/shap-e)

In [ ]:
#@title <font color="#97C145">**1. Clonar el repositorio oficial e instalar dependencias.**</font>
#@markdown ---
#@markdown Puede tardar un rato en instalarse, no te preocupes, no se ha quedado pillado.
%%capture
!git clone https://github.com/openai/shap-e
%cd /content/shap-e/
!pip install -e .

In [ ]:
#@title <font color="#97C145">**2. Sube tu imagen.**
#@markdown ---
#@markdown Sube una imagen para hacer la conversión de IMG a 3D. (Convenientemente .png transparente para más claridad.)
from google.colab import files
import os

def subir_archivo():
    uploaded = files.upload()
    for nombre_archivo, contenido_archivo in uploaded.items():
        ruta_archivo = os.path.join('/content', nombre_archivo)
        with open(ruta_archivo, 'wb') as archivo:
            archivo.write(contenido_archivo)
        print(f"Archivo subido: {ruta_archivo}")

subir_archivo()

In [ ]:
#@title <font color="#97C145">**3. Ejecutar**
#@markdown Remplaza donde pone "tufoto" por el nombre del archivo que has subido.
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

batch_size = 4
guidance_scale = 15.0

batch_size = 4
guidance_scale = 3.0
image_directory = "/content/tufoto.png" #@param {type:"string"}
image = load_image(image_directory)

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

render_mode = 'nerf' # you can change this to 'stf'
#
size = 128 #@param {type:"number"} 

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)